In [1]:
import tensorflow as tf
import numpy as np
import collections
import os
import argparse
import datetime as dt

In [2]:
data_path = "/Users/kevindsouza/Documents/UBC/Term2/CPSC532L/Project/TensorFlow/graphs-and-NLP/LSTM/LSTM/simple-examples/data"

In [3]:
#parser = argparse.ArgumentParser()
#parser.add_argument('run_opt',type=int, default=2, help='A integer: 1 to train, 2 to test')
#parser.add_argument('--data_path',type=str,default=data_path,help='the full path of training data')
#args = parser.parse_args()

In [4]:
def read_words(filename):
    with tf.gfile.GFile(filename,"r") as f:
        return f.read().replace("\n","<eos>").split()
    
def build_vocab(filename):
    data = read_words(filename)
    
    #print(data[1])
    counter = collections.Counter(data)
    #print(counter)
    count_pairs = sorted(counter.items(),key=lambda x: (-x[1],x[0]))
    #print(count_pairs)
    
    words,_ = list(zip(*count_pairs))
    
    word_to_id = dict(zip(words,range(len(words))))
    return word_to_id

def file_to_word_ids(filename,word_to_id):
    data = read_words(filename)
    return [word_to_id[word] for word in data if word in word_to_id]

def load_data():
    train_path = os.path.join(data_path,"ptb.train.txt")
    valid_path = os.path.join(data_path,"ptb.valid.txt")
    test_path = os.path.join(data_path,"ptb.test.txt")
    
    #build vocab and then convert into list of integers
    word_to_id = build_vocab(train_path)
    train_data = file_to_word_ids(train_path,word_to_id)
    valid_data = file_to_word_ids(valid_path,word_to_id)
    test_data = file_to_word_ids(test_path,word_to_id)
    
    vocabulary = len(word_to_id)
    
    reversed_dictionary = dict(zip(word_to_id.values(),word_to_id.keys()))
    
    print(train_data[:5])
    print(vocabulary)
    #print(word_to_id)
    print(" ".join([reversed_dictionary[x] for x in train_data[:10]]))
    
    return train_data,valid_data,test_data,vocabulary,reversed_dictionary 


train_data,valid_data,test_data,vocabulary,reversed_dictionary = load_data()                   

[9970, 9971, 9972, 9974, 9975]
10000
aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec


In [5]:
def batch_producer(raw_data,batch_size,num_steps):
    raw_data = tf.convert_to_tensor(raw_data,name="raw_data",dtype=tf.int32)
    
    data_len = tf.size(raw_data)
    batch_len = data_len // batch_size
    
    data = tf.reshape(raw_data[0:batch_size*batch_len],[batch_size,batch_len])
    
    epoch_size = (batch_len - 1) // num_steps
    
    i = tf.train.range_input_producer(epoch_size, shuffle=False).dequeue()
    #print(i)
    x = data[:,i*(num_steps):(i+1)*num_steps]
    #print(x)
    x.set_shape([batch_size,num_steps])
    y = data[:,i*(num_steps) + 1:(i+1)*num_steps + 1]
    y.set_shape([batch_size,num_steps])
    
    return x,y


In [6]:
class Input(object):
    def __init__(self,batch_size,num_steps,data):
        self.batch_size = batch_size
        self.num_steps = num_steps
        self.epoch_size = ((len(data)//batch_size)-1) // num_steps
        self.input_data,self.targets = batch_producer(data,batch_size,num_steps)
        
        
        

In [7]:
class Model(object):
    def __init__(self,input,is_training,hidden_size,vocab_size,num_layers,dropout=0.5,init_scale=0.05):
        self.is_training = is_training
        self.input_obj = input 
        self.batch_size = input.batch_size
        self.num_steps = input.num_steps 
        self.hidden_size = hidden_size 
        
        
        #create the word embeddings 
        with tf.device("/cpu:0"):
            embedding = tf.Variable(tf.random_uniform([vocab_size,self.hidden_size],-init_scale,init_scale))
            inputs = tf.nn.embedding_lookup(embedding,self.input_obj.input_data)
        
        #print(inputs)
        
        if is_training and dropout < 1:
            inputs = tf.nn.dropout(inputs,dropout)
            
            
        
        #set up state storage and extraction
        self.init_state = tf.placeholder(tf.float32,[num_layers,2,self.batch_size,self.hidden_size])
        
        #print(self.init_state)
        state_per_layer_list = tf.unstack(self.init_state,axis=0)
        rnn_tuple_state = [tf.contrib.rnn.LSTMStateTuple(state_per_layer_list[idx][0],
                                 state_per_layer_list[idx][1]) for idx in range(num_layers)]
        
        #print(inputs)
        print(rnn_tuple_state)
        #create LSTM cell to be unrolled 
        cell = tf.contrib.rnn.LSTMCell(hidden_size,forget_bias=1.0,state_is_tuple=True)
        
        if is_training and dropout < 1:
            cell = tf.contrib.rnn.DropoutWrapper(cell,output_keep_prob = dropout)
        if num_layers > 1:
            cell = tf.contrib.rnn.MultiRNNCell([cell for _ in range(num_layers)], state_is_tuple = True) 
            
        output,self.state = tf.nn.dynamic_rnn(cell,inputs,initial_state = rnn_tuple_state, dtype=tf.float32)
                        
        
        output = tf.reshape(output,[-1,hidden_size])
        
        softmax_w = tf.Variable(tf.random_uniform[hidden_size,vocab_size],-init_scale,init_scale)
        softmax_b = tf.Variable(tf.random_uniform[vocab_size],-init_scale,init_scale)
        
        logits = tf.nn.xw_plus_b(output,softax_w,softmax_b)
        
        #reshape logits to be a 3D tensor for sequence loss
        logits = tf.reshape(logits,[self.batch_size,self.num_steps,vocab_size])
        
        #use the contrib sequence loss and average over the batches 
        loss = tf.contrib.seq2seq.sequence_loss(logits,self.input_obj.targets,
                                               tf.ones([self.batch_size,self.num_steps],dtype=np.float32),
                                                average_across_timesteps = False,
                                                average_across_batch = True)
        #update cost 
        self.cost = tf.reduce_sum(loss)
        
        #get prediction accuracy
        self.softmax_out = tf.nn.softmax(tf.reshape(logits,[-1,vocab_size]))
        self.predict = tf.cast(tf.argmax(self.softmax_out,axis=1),tf.int32)
        correct_prediction = tf.equal(self.predict,tf.reshape(self.input_obj.targets,[-1]))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        
        if not is_training:
            return 
        
        
        
        self.learning_rate = tf.Variable(0.0,trainable = False)
        
        tvars = tf.trainable_variables()
        grads,_ = tf.clip_by_global_norms(tf.gradients(self.cost,tvars),5)
        optimizer = tf.train.GradientDescentOptimizer(self.learning_rate)
        
        self.train_op = optimizer.applygradients(zip(grads,tvars), 
                                                 global_step = tf.contrib.framework.get_or_create_global_step())
        
        self.new_lr = tf.placeholder(tf.floa32,[])
        self.lr_update = tf.assign(self.learning_rate,self.new_lr)
        
        
    def assign_lr(self,session,lr_value):
        
        session.run(self.lr_update, feed_dict={self.new_lr:lr_value})
        
        

In [8]:
def train(train_data,vocabulary,num_layers,num_epochs,batch_size,model_save_name,learning_rate=1.0,
          max_lr_epoch=10,lr_decay=0.93,print_iter=50):
    
    training_input = Input(batch_size=batch_size, num_steps=35, data=train_data)
    print(training_input.input_data)
    print(training_input.targets)
    
    m = Model(training_input,is_training = True,hidden_size=650,vocab_size=vocabulary,
             num_layers=num_layers)
    
    init_op = tf.global_variables_initializer()
    orig_decay = lr_decay
    
    with tf.Session() as sess:
        sess.run([init_op])
        coord = tf.train.Coordinator() 
        threads = tf.train.start_queue_runners(coord=coord)
        saver = tf.train.Saver()
        
        for epoch in range(num_epochs):
            new_lr_decay = orig_decay ** max(epoch+1-max_lr_epoch,0.0)        
            m.assign_lr(sess,learning_rate*new_lr_decay)
            
            current_state = np.zeros((num_layers,2,batch_size,m.hidden_size))
            curr_time = dt.datetime.now()
            
            for step in range(training_input.epoch_size):
                if step % print_iter != 0:
                    cost,_,current_state = sess.run([m.cost,m.train_op,m.state],
                                                    feed_dict={m.init_state:current_state})
                    
                else:
                    seconds = (float((dt.datetime.now() - curr_time).seconds) // print_iter)
                    curr_time = dt.datetime.now()
                    cost,_,current_state,acc = sess.run([m.cost,m.train,m.state,m.accuracy],
                                                       feed_dict={m.init_state:current_state})
                    print("Epoch {}, step {}, cost {:.3f},accuracy {:,3f},seconds_per_step {:.3f}".format(
                    epoch,step,cost,acc,seconds))
                    
            saver.save(sess, data_path + '\\' + model_save_name, global_step=epoch)
            
        saver.save(sess, data_path + '\\' + model_save_name + '-final')
        
        #close threads
        coord.request_stop()
        coord.join(threads)
        
                
                
        

In [9]:
num_layers = 1
num_epochs = 2
batch_size = 50
model_save_name = "lstm"

train(train_data,vocabulary,num_layers,num_epochs,batch_size,model_save_name)

Tensor("strided_slice_1:0", shape=(50, 35), dtype=int32)
Tensor("strided_slice_2:0", shape=(50, 35), dtype=int32)
Tensor("embedding_lookup:0", shape=(50, 35, 650), dtype=float32, device=/device:CPU:0)
Tensor("dropout/mul:0", shape=(50, 35, 650), dtype=float32)
[LSTMStateTuple(c=<tf.Tensor 'strided_slice_3:0' shape=(50, 650) dtype=float32>, h=<tf.Tensor 'strided_slice_4:0' shape=(50, 650) dtype=float32>)]


ValueError: not enough values to unpack (expected 2, got 1)